## Tumor Growth Simulation
### Darpan Gaur : CO21BTECH11004
### Yoshita Kondapalli : CO21BTECH11008

The code written below is the modification and extension of the code for Cahn-Hilliard equation, a nonlinear, time-dependent fourth-order PDE [link](https://docs.fenicsproject.org/dolfinx/main/python/demos/demo_cahn-hilliard.html).


In [ ]:
# import libraries
from dolfin import *
import random
import numpy as np
import time
from mshr import *  # For complex meshes
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from os import path
from shutil import copyfile
from plotSaveUtils import *
import warnings
from ffc.quadrature.deprecation import QuadratureRepresentationDeprecationWarning
warnings.simplefilter("once", QuadratureRepresentationDeprecationWarning)

### Initialization

In [ ]:
class SimulationParameters:
    """ Mesh parameters """
    dimension = 2  # Fixed to 2D
    domain_length = 12.5  # Length of mesh
    mesh_resolution = 64  # Number of grid points
    
    """ FE function parameters """
    quadrature_degree = 4  # Degree of quadrature
    mass_lumping = True  # Mass lumping (only for polynomial_degree=1)
    polynomial_degree = 1  # Degree of polynomials
    
    """ Time parameters """
    time_step = 1.0e-03  # Time step
    fix_time_interval = True  # Use total_time if True, or num_time_steps if False
    num_time_steps = 5  # Number of time iterations
    total_time = 25  # Final time
    
    def __init__(self):
        if self.fix_time_interval:
            self.num_time_steps = int(self.total_time/self.time_step)
        else:
            self.total_time = self.num_time_steps*self.time_step
    
    """ Parameters for mesh adaptation """
    adapt = True
    adapt_delta = 0.05  # Refine where |phi|< 1-adapt_delta
    adapt_freq = 10  # Coarse mesh every adapt_freq steps
    adapt_coarse_size = 64  # Coarsest mesh
    adapt_fine_size = 2**10  # Finest mesh
    adapt_max_iterations = 5  # Max refinement iterations

sim_params = SimulationParameters()

In [ ]:
class OutputParameters:
    
    """ Save output """
    save_as_vtu = True  # For paraview visualization
    output_path = f"simulation_output"
    output_freq = 100  # Iteration frequency for saving output
    
    """ Plot during run """
    plot_init = False  # Plot initial conditions?
    plot_solution = False  # Plot solutions in real time?
    plot_freq = 50  # Plot frequency
    
    """ Boundary conditions """
    boundary_x0 = 0
    boundary_x1 = 2
    boundary_y0 = 0
    boundary_y1 = 2

output_params = OutputParameters()

class MyNonlinearProblem(NonlinearProblem):
    def __init__(self, a, L, bc=None):
        NonlinearProblem.__init__(self)
        self.L = L
        self.a = a
        self.bc = bc
        
    def F(self, b, x):
        assemble(self.L, tensor=b)
        if self.bc is not None:  # Optional Dirichlet boundary conditions
            self.bc.apply(b, x)
            
    def J(self, A, x):
        assemble(self.a, tensor=A)
        if self.bc is not None:  # Optional Dirichlet boundary conditions
            self.bc.apply(A)

### Boundary Conditions

In [ ]:
class DirichletBoundary(SubDomain):
    def inside(self, x, on_boundary):

        """ Rectangle mesh """
        is_boundary = False
        tol = 1e-14
        
        # X boundaries
        if output_params.boundary_x0 == 1:
            is_boundary = is_boundary or near(x[0], 0.0, tol)
        if output_params.boundary_x1 == 1:
            is_boundary = is_boundary or near(x[0], sim_params.domain_length, tol)
            
        # Y boundaries
        if output_params.boundary_y1 == 1:
            is_boundary = is_boundary or near(x[1], sim_params.domain_length, tol)
        if output_params.boundary_y0 == 1:
            is_boundary = is_boundary or near(x[1], 0.0, tol)
            
        return (is_boundary and on_boundary)

In [ ]:
class BoundaryX0(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[0], 0, DOLFIN_EPS)

class BoundaryX1(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[0], sim_params.domain_length, DOLFIN_EPS)

class BoundaryY0(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[1], 0, DOLFIN_EPS)

class BoundaryY1(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and near(x[1], sim_params.domain_length, DOLFIN_EPS)

### Tumor Growth mathaematical model

In [ ]:
class TumorModel:
    def __init__(self):
        self.num_equations = 3  # Number of equations in the system
        # self.interface_width = 0.1  # Interface width parameter
        self.interface_width = 0.01
        self.surface_tension = 0.1  # Surface tension
        self.proliferation_rate = 0.1  # Proliferation rate
        self.apoptosis_rate = DOLFIN_EPS  # Apoptosis rate
        self.consumption_rate = 1.0  # Consumption rate
        self.diffusion_param = 0.02  # Diffusion parameter
        self.chemotaxis = 5.0  # Chemotaxis
        
        self.sigma_boundary = 1.0  # Dirichlet boundary value for sigma
        self.sigma_infinity = 1.0  # Robin boundary conditions
        self.robin_constant = 1000.0
        
        self.sigma_initial = 1.0  # Initial condition for sigma
        self.dynamic = True  # Dynamic or quasi-static equation for sigma
        
        self.boundary_conditions = {
            1: {'Dirichlet': self.sigma_boundary},  # sig = sig_B
            2: {'Robin': (self.robin_constant, self.sigma_infinity)},  # del_n sig = K(sig_inf - sig)
            0: {'Neumann': 0}  # del_n sig = 0
        }
    
    def init_problem(self, time_step, u, u0):
        """ Variables """
        ME = u.function_space()
        mesh = u.function_space().mesh()
        du = TrialFunction(ME)
        q, v, w = TestFunctions(ME)
        dc, dmu, dsig = split(du)
        c, mu, sig = split(u)
        c0, mu0, sig0 = split(u0)
        
        """ Nonlinear functions """
        c = variable(c)
        DPsi = (c**3-c0)  # Derivative of potential, dpsi/dphi
        m_phi = 0.5*(1+c0)**2 + 5e-06  # Mobility function m(phi)
        h_phi = 0.5*(1+c0)  # Interpolation function h(phi)
        
        """ First equation, testing with q """
        L01 = c*q*dx - c0*q*dx      
        L02 = time_step*m_phi*dot(grad(mu), grad(q))*dx
        L03 = -time_step*(self.proliferation_rate*sig-self.apoptosis_rate)*h_phi*q*dx
        L0 = L01 + L02 + L03
        
        """ Second equation, testing with v """
        L11 = mu*v*dx - self.surface_tension/self.interface_width*DPsi*v*dx
        L12 = -self.surface_tension*self.interface_width*dot(grad(c), grad(v))*dx
        L13 = self.chemotaxis*sig*v*dx
        L1 = L11 + L12 + L13
        
        """ Third equation, testing with w """
        L21 = dot(grad(sig), grad(w))*dx
        if self.dynamic:  # Dynamic or quasi-static
            L21 += 1./time_step*(sig*w*dx - sig0*w*dx)
        L22 = -self.diffusion_param*dot(grad(c), grad(w))*dx
        L23 = self.consumption_rate*sig*h_phi*w*dx
        L2 = L21 + L22 + L23
        
        """ Set boundary conditions for sigma """
        boundary_markers = MeshFunction("size_t", mesh, mesh.topology().dim()-1, 9999)
        boundary_markers.set_all(9999)
        bx0 = BoundaryX0()
        bx1 = BoundaryX1()
        by0 = BoundaryY0()
        by1 = BoundaryY1()
        
        bx0.mark(boundary_markers, output_params.boundary_x0)
        bx1.mark(boundary_markers, output_params.boundary_x1)
        by0.mark(boundary_markers, output_params.boundary_y0)
        by1.mark(boundary_markers, output_params.boundary_y1)
        
        dirichlet = False
        dirichlet = dirichlet or (output_params.boundary_x0==1) or (output_params.boundary_x1==1)
        dirichlet = dirichlet or (output_params.boundary_y0==1) or (output_params.boundary_y1==1)
        
        # Redefine boundary integration measure
        ds = Measure('ds', domain=mesh, subdomain_data=boundary_markers)
        
        # Collect Neumann integrals
        for i in self.boundary_conditions:
            if 'Neumann' in self.boundary_conditions[i]:
                if self.boundary_conditions[i]['Neumann'] != 0:
                    g = self.boundary_conditions[i]['Neumann']
                    L2 += g*w*ds(i)
        
        # Collect Robin integrals
        for i in self.boundary_conditions:
            if 'Robin' in self.boundary_conditions[i]:
                K, sig_inf = self.boundary_conditions[i]['Robin']
                L2 += K*(sig-sig_inf)*w*ds(i)
        
        # Define Dirichlet boundary conditions
        bc = None
        if dirichlet:
            bc = DirichletBC(ME.sub(2), self.sigma_boundary, DirichletBoundary())
        
        """ Compute directional derivative (Jacobian) """
        L = L0 + L1 + L2
        a = derivative(L, u, du)
        
        """ Create nonlinear problem """
        if dirichlet:
            problem = MyNonlinearProblem(a, L, bc)
        else:
            problem = MyNonlinearProblem(a, L)
        
        return problem

tumor_model = TumorModel()

### Initialization

In [ ]:
def initial_tumor_shape(x):
    """Define initial tumor shape as a circle in the domain center"""
    if x[0]<DOLFIN_EPS:
        phi = np.pi/2
    else:
        phi = np.arctan(x[1]/x[0])
    r = 0
    for i in range(0,len(x)):
        r += x[i]**2
    r = r**(1./2)
    r += -(2+0.1*cos(2*phi))
    value = -np.tanh(r/(sqrt(2)*tumor_model.interface_width))
    return value

class InitialConditions(UserExpression):
    def __init__(self, **kwargs):
        random.seed(2 + MPI.rank(MPI.comm_world))
        super().__init__(**kwargs)
    
    def value_shape(self):
        return (tumor_model.num_equations,)
    
    def eval(self, values, x):
        values[0] = initial_tumor_shape(x)
        values[1] = 0.0  # Initial chemical potential
        values[2] = tumor_model.sigma_initial  # Initial nutrient concentration

In [ ]:
def create_mesh(N=None):
    """Create a 2D mesh"""
    if N is None:
        N = sim_params.mesh_resolution
    
    length = sim_params.domain_length
    
    mesh = RectangleMesh.create(MPI.comm_world, 
                                [Point(0,0), Point(length,length)], 
                                [N,N], 
                                CellType.Type.triangle, 
                                "crossed")
    
    return mesh

def refine_mesh(u_old, j):
    """Adaptively refine mesh based on the phase field interface"""
    # Check if u_old is a function or initial condition
    if isinstance(u_old, Function):
        init = False
    else:
        init = True
    
    # Refine old mesh or create new mesh
    if (j+sim_params.adapt_freq-1)%sim_params.adapt_freq>0 and not init:
        mesh_tmp = u_old.function_space().mesh()
    else:
        mesh_tmp = create_mesh(sim_params.adapt_coarse_size)
    
    # Create temporary function space
    V_tmp = FunctionSpace(mesh_tmp, "Lagrange", sim_params.polynomial_degree)
    u_tmp = Function(V_tmp)
    
    # Interpolate solution onto temporary mesh
    if init:
        P1_tmp = FiniteElement("Lagrange", mesh_tmp.ufl_cell(), sim_params.polynomial_degree)
        ME_tmp = FunctionSpace(mesh_tmp, MixedElement(tumor_model.num_equations*[P1_tmp]))
        u_init_tmp = Function(ME_tmp)
        u_init_tmp.interpolate(u_old)
        u_tmp.interpolate(u_init_tmp.split()[0])
    else:
        u_tmp.interpolate(u_old.split()[0])
    
    # Minimum edge length for refinement
    hmin = sim_params.domain_length / sim_params.adapt_fine_size * 1.000001
    
    # Iteratively refine mesh
    iteration = 0
    max_iterations = sim_params.adapt_max_iterations
    continue_refinement = True
    
    while continue_refinement and iteration < max_iterations:
        iteration += 1
        markers = MeshFunction("bool", mesh_tmp, mesh_tmp.topology().dim()-1)
        markers.set_all(False)
        no_refinement_needed = True
        
        # Mark edges for refinement
        for facet in facets(mesh_tmp):
            p = facet.midpoint()
            h = min([edge.length() for edge in edges(facet)])
            # Mark facets near the interface for refinement
            if u_tmp(p)<1-sim_params.adapt_delta and u_tmp(p)>-1+sim_params.adapt_delta and h>=hmin-DOLFIN_EPS:
                markers[facet] = True
                no_refinement_needed = False
        
        continue_refinement = not no_refinement_needed
        
        if continue_refinement:
            mesh_tmp = refine(mesh_tmp, markers)
            print(f"Refined mesh: max h = {mesh_tmp.hmax()}, min h = {mesh_tmp.hmin()}")
            
            # Update function on refined mesh
            V_tmp = FunctionSpace(mesh_tmp, "Lagrange", sim_params.polynomial_degree)
            u_tmp = Function(V_tmp)
            
            if init:
                P1_tmp = FiniteElement("Lagrange", mesh_tmp.ufl_cell(), sim_params.polynomial_degree)
                ME_tmp = FunctionSpace(mesh_tmp, MixedElement(tumor_model.num_equations*[P1_tmp]))
                u_init_tmp = Function(ME_tmp)
                u_init_tmp.interpolate(u_old)
                u_tmp.interpolate(u_init_tmp.split()[0])
            else:
                u_tmp.interpolate(u_old.split()[0])
    
    return mesh_tmp

In [ ]:
def init_function_spaces(mesh, u_old=None, u0_old=None):
    """Initialize function spaces on the given mesh"""
    P1 = FiniteElement("Lagrange", mesh.ufl_cell(), sim_params.polynomial_degree)
    ME = FunctionSpace(mesh, MixedElement(tumor_model.num_equations*[P1]))
    u = Function(ME)
    u0 = Function(ME)
    
    if u_old is not None:
        u.interpolate(u_old)
        if u0_old is not None:
            u0.interpolate(u0_old)
    
    return (u, u0)

### main function

In [ ]:
def main():
    """Main function to run the simulation"""
    # Initialize form compiler
    setup_form_compiler()
    
    # MPI setup
    # my_rank = MPI.rank(MPI.comm_world)
    
    # Print simulation information
    print("\nStarting 2D tumor growth simulation")
    print(f"Domain size: {sim_params.domain_length} x {sim_params.domain_length}")
    print(f"Mesh resolution: {sim_params.mesh_resolution}")
    print(f"Time step: {sim_params.time_step}")
    print(f"Total simulation time: {sim_params.total_time}")
    print(f"Number of time steps: {sim_params.num_time_steps}\n")
    
    u_init = InitialConditions(degree=sim_params.polynomial_degree)
    

    # Create mesh, with optional adaptation
    if sim_params.adapt:
        mesh = refine_mesh(u_init, 0)
    else:
        mesh = create_mesh()
    print(f"Mesh created with {mesh.num_cells()} cells")
    
    # Initialize functions
    u, u0 = init_function_spaces(mesh)
    u.interpolate(u_init)
    
    # Initialize problem
    problem = tumor_model.init_problem(sim_params.time_step, u, u0)
    solver = configure_newton_solver()
    print(f"Initialization completed")

    # Initialize time
    j = 0
    t = 0
    
    # Open output files
    file0, file1, file2 = open_output_files(output_params, tumor_model)
    
    # Save initial state
    save_vtu_output(output_params, tumor_model, sim_params, u, j, t, file0, file1, file2)
    
    # Time iteration
    while (j < sim_params.num_time_steps):
        j += 1
        t += sim_params.time_step
        
        # Solve current time step
        u0.vector()[:] = u.vector()
        solver.solve(problem, u.vector())
        
        print(f"\nCompleted iteration {j} of {sim_params.num_time_steps}\n")
        
        # Save results
        save_vtu_output(output_params, tumor_model, sim_params, u, j, t, file0, file1, file2)
        
        # Adapt mesh if needed
        if sim_params.adapt:
            mesh = refine_mesh(u, j)
            u, u0 = init_function_spaces(mesh, u, u0)
            problem = tumor_model.init_problem(sim_params.time_step, u, u0)
            solver = configure_newton_solver()


In [ ]:
if __name__ == "__main__":
    main()